In [1]:
import os
import geopandas as gpd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")
local_crs = 32636
# path to data
example_data_path = "./data"

## Input data fetch and parameters setting

In [4]:
from masterplan_tools.method.blocks import CutParameters

city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet")).to_crs(local_crs)
water_geometry = gpd.read_parquet(os.path.join(example_data_path, "water_geometry.parquet")).to_crs(local_crs)
roads_geometry = gpd.read_parquet(os.path.join(example_data_path, "roads_geometry.parquet")).to_crs(local_crs)
railways_geometry = gpd.read_parquet(os.path.join(example_data_path, "railways_geometry.parquet")).to_crs(local_crs)

# basically we need just these 4 geometries to cut some blocks
cut_params = CutParameters(
  city=city_geometry,
  water=water_geometry,
  roads=roads_geometry,
  railways=railways_geometry
)

To improve our method we should use land use filtering. If we don't set landuse parameters, no LU filtering will be applied to the blocks

In [7]:
from masterplan_tools.method.blocks import LandUseParameters

no_development = gpd.read_file(os.path.join(example_data_path, "no_development_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
no_development = no_development[no_development['RAYON']=='Василеостровский']
landuse = gpd.read_file(os.path.join(example_data_path, "landuse_zone_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
buildings_geom = gpd.read_file(os.path.join(example_data_path, "buildings_blocks.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)



In [8]:
lu_params = LandUseParameters(
  no_development=no_development,
  landuse=landuse,
  buildings=buildings_geom
)

To generate city blocks GeoDataFrame we use the `BlockCutter` class

In [ ]:
from masterplan_tools.method import BlocksCutter

blocks = BlocksCutter(
  cut_parameters=cut_params, 
  lu_parameters=lu_params,
).get_blocks()

In [10]:
blocks.to_gdf().explore()

In [ ]:
blocks.to_gdf().head()

There are three landuse tags in the blocks gdf:
1. 'no_dev_area' -- according to th no_debelopment gdf and cutoff without any buildings or specified / selected landuse types;
2. 'selected_area' -- according to the landuse gdf. We separate theese polygons since they have specified landuse types;
3. 'buildings' -- there are polygons that have buildings landuse type. 

In further calculations we will use the in the following steps:
- Only 'buildings' -- to find clusters of buildings in big polygons;
- All of them while calculating the accessibility times among city blocks;
- All of them except 'no_dev_area' while optimizing the development of new facilities.

In [ ]:
blocks.to_gdf().plot(column='landuse', legend=True)

Save the result for the next example

In [ ]:
blocks.to_gdf().to_parquet(os.path.join(example_data_path, "blocks_cutter_result.parquet"))